In [1]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


In [2]:
# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

DEBUG:lib.lab_lib:hello


# 2022年5月28日

LULESHのプロファイルから予測をする

In [3]:
date :str = "2022年6月1日"

input_rawDF :pd.DataFrame = return_rawDFinLULESH(
    processes=lulesh_processes,
    iterations=lulesh_iterations,
    sizes=lulesh_sizes,
    csvDirPath=csvDirPath,
)

input_rawDF = input_rawDF.rename(columns={"Name":"functionName"})

input_list_expVarNames :list[str] = ["process", "iteration", "size"]
input_list_resVarNames :list[str] = ["#Call"]

model_names: list[str] = [
    "modelLin",
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations"
    # "modelBasicTree",
]

# 集合演算を用いて説明変数・目的変数が入力DFに存在していることを確認
if input_list_expVarNames == [] or (set(input_list_expVarNames).issubset(set(input_rawDF.columns.tolist())) == False):
    warnings.warn(f"説明変数の指定に誤りがあります")
    # return False
if input_list_resVarNames == [] or (set(input_rawDF.columns.tolist()) == False):
    warnings.warn(f"問題変数の指定に誤りがあります")
    # return False

function_names :list[str] = list(set(input_rawDF["functionName"].tolist()))
if len(function_names) == 0:
    warnings.warn("与えられたDFに関数名がありません")
    # return False

result_series_list :list[pd.Series] = []

for function_name in function_names:
    # 関数ごとの生データ
    input_rawDF_per_function = input_rawDF[input_rawDF["functionName"] == function_name]
    # モデルの構築
    models = Models(
        inputDF = input_rawDF_per_function,
        expVarColNames=input_list_expVarNames,
        resVarColNames=input_list_resVarNames,
        targetDF=None,
        modelNames=model_names,
    )

    models.setUpDataBeforeCalcLr()
    models.calcLr()
    models.calcMAPE()

    dictCalcedMAPE = models.returnCalculatedMAPE()

    # 算出されたMAPEの数値をfloatにする
    for key in dictCalcedMAPE.keys():
        dictCalcedMAPE[key] = float(dictCalcedMAPE[key])

    # 関数ごとの結果に格納
    dict_for_series :dict = copy.deepcopy(dictCalcedMAPE)
    dict_for_series["functionName"] = function_name

    series = pd.Series(dict_for_series)
    result_series_list.append(series)

resultDF :pd.DataFrame = pd.DataFrame(result_series_list)
resultDF = addLowestMAPEColumn(inputDF=resultDF, model_name_list=model_names, version=2)


/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estima

In [4]:
resultDF

,modelLin,modelIp,modelLog,modelProcessDividedByProblemSize,modelProblemSizeDividedByProcess,modelLinAndIp,modelLinAndLog,modelIpAndLin,modelIpAndLog,modelLogAndLin,modelLogAndIp,modelLinearSumOf2elementCombination,modelLinearSumOfElementCombinations,functionName,最低値
0,1.646390e-13,177.412047,115.178571,7.668323e+01,7.655747e+01,177.412047,115.178571,9.075633e-15,115.178571,4.070818e-14,177.412047,117.100765,2.199475e-12,void_CommSyncPosVel(Domain,9.075633e-15
1,3.085659e+02,0.000074,243.777234,1.808193e+02,6.014111e+01,308.565885,308.565885,7.382729e-05,0.000074,2.437772e+02,243.777234,346.822183,1.079287e-04,void_VerifyAndWriteFinalOutput(Real_t_Domain,7.382729e-05
2,0.000000e+00,0.000000,0.000000,2.308030e-08,1.862710e-08,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,void_Domain::SetupBoundaryConditions(Int_t),0.000000e+00
3,0.000000e+00,0.000000,0.000000,2.308030e-08,1.862710e-08,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,void_Domain::SetupSymmetryPlanes(Int_t),0.000000e+00
4,0.000000e+00,0.000000,0.000000,2.308030e-08,1.862710e-08,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,MPI_Finalize(),0.000000e+00
5,1.646390e-13,177.412047,115.178571,7.668323e+01,7.655747e+01,177.412047,115.178571,9.075633e-15,115.178571,4.070818e-14,177.412047,117.100765,2.199475e-12,void_CommMonoQ(Domain,9.075633e-15
6,0.000000e+00,0.000000,0.000000,2.308030e-08,1.862710e-08,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,.TAU_application,0.000000e+00
7,0.000000e+00,0.000000,0.000000,2.308030e-08,1.862710e-08,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,MPI_Reduce(),0.000000e+00
8,0.000000e+00,0.000000,0.000000,2.308030e-08,1.862710e-08,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,void_Domain::~Domain(),0.000000e+00
9,9.897074e+03,11008.061076,12824.894431,1.406233e+02,1.407564e+02,11008.061076,12824.894431,9.897074e+03,12824.894431,9.897074e+03,11008.061076,4521.029402,1.327401e+03,Real_t_CalcElemVolume(const,1.406233e+02


In [5]:
resultDF.mean()

/tmp/ipykernel_11263/2794288614.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  resultDF.mean()


modelLin                               324.173667
modelIp                                411.497704
modelLog                               454.526334
modelProcessDividedByProblemSize        40.081234
modelProblemSizeDividedByProcess        35.645728
modelLinAndIp                          421.093841
modelLinAndLog                         455.735629
modelIpAndLin                          315.660772
modelIpAndLog                          446.710080
modelLogAndLin                         323.795580
modelLogAndIp                          419.026472
modelLinearSumOf2elementCombination    194.468150
modelLinearSumOfElementCombinations     41.843775
最低値                                      4.756974
dtype: float64

In [6]:
DF_funcName_lowestMAPE :pd.DataFrame = resultDF.filter(items=["functionName", "最低値"])
DF_funcName_lowestMAPE = DF_funcName_lowestMAPE.rename(columns={"functionName":"関数名", "最低値":"最低MAPE"})

print(DF_funcName_lowestMAPE.style.format(precision=2).to_latex(
    caption=f"ベンチマークプログラムLULESHの関数における最低MAPE",
    label=f"{date}lowestMAPEinLULESH"
))

\begin{table}
\caption{ベンチマークプログラムLULESHの関数における最低MAPE}
\label{2022年6月1日lowestMAPEinLULESH}
\begin{tabular}{llr}
 & 関数名 & 最低MAPE \\
0 & void_CommSyncPosVel(Domain & 0.00 \\
1 & void_VerifyAndWriteFinalOutput(Real_t_Domain & 0.00 \\
2 & void_Domain::SetupBoundaryConditions(Int_t) & 0.00 \\
3 & void_Domain::SetupSymmetryPlanes(Int_t) & 0.00 \\
4 & MPI_Finalize() & 0.00 \\
5 & void_CommMonoQ(Domain & 0.00 \\
6 & .TAU_application & 0.00 \\
7 & MPI_Reduce() & 0.00 \\
8 & void_Domain::~Domain() & 0.00 \\
9 & Real_t_CalcElemVolume(const & 140.62 \\
10 & MPI_Isend() & 3.87 \\
11 & MPI_Comm_size() & 0.00 \\
12 & MPI_Allreduce() & 0.00 \\
13 & void_CommSend(Domain & 0.00 \\
14 & void_Domain::CreateRegionIndexSets(Int_t_Int_t) & 0.00 \\
15 & void_CalcKinematicsForElems(Domain & 0.00 \\
16 & MPI_Barrier() & 0.00 \\
17 & MPI_Waitall() & 0.00 \\
18 & MPI_Init() & 0.00 \\
19 & void_Domain::SetupCommBuffers(Int_t) & 0.00 \\
20 & void_InitMeshDecomp(Int_t_Int_t_Int_t & 0.00 \\
21 & MPI_Wait() & 3.87 \\


# 2022年5月25日～

重み付き平均の実装

todos
* ✅✅"modelLin":ModelLinForMultipleRegression
* ✅✅"modelIp":ModelIpForMultipleRegression
* ✅✅"modelLog":ModelLogForMultipleRegression
* ✅✅"modelLinAndIp":ModelMultipleEquationForMultipleRegression
* ✅✅"modelLinAndLog":ModelMultipleEquationForMultipleRegression
* ✅✅"modelIpAndLin":ModelMultipleEquationForMultipleRegression
* ✅✅"modelIpAndLog":ModelMultipleEquationForMultipleRegression
* ✅✅"modelLogAndLin":ModelMultipleEquationForMultipleRegression
* ✅✅"modelLogAndIp":ModelMultipleEquationForMultipleRegression
* ✅✅"modelProcessDividedByProblemSize":Model_ProcessesDevidedByProblemSize_ForMultipleRegression
* ✅✅"modelProblemSizeDividedByProcess":Model_ProblemSizeDevidedByProcesses_ForMultipleRegression
* ✅✅"modelLinearSumOf2elementCombination":Model_LinearSumOf2elementCombination_ForMultipleRegression
* ✅✅"modelLinearSumOfElementCombinations":Model_LinearSumOfElementCombinations_ForMultipleRegression

* `class Models`での対応
    * ✅models.calcWeightedMAPE()
    * ✅models.returnCalculatedMAPE()

In [7]:
date :str = "2022年6月1日"

input_rawDF :pd.DataFrame = return_rawDFinLULESH(
    processes=lulesh_processes,
    iterations=lulesh_iterations,
    sizes=lulesh_sizes,
    csvDirPath=csvDirPath,
)

input_rawDF = input_rawDF.rename(columns={"Name":"functionName"})

input_list_expVarNames :list[str] = ["process", "iteration", "size"]
input_list_resVarNames :list[str] = ["#Call"]

model_names: list[str] = [
    "modelLin",
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations"
    # "modelBasicTree",
]

# 集合演算を用いて説明変数・目的変数が入力DFに存在していることを確認
if input_list_expVarNames == [] or (set(input_list_expVarNames).issubset(set(input_rawDF.columns.tolist())) == False):
    warnings.warn(f"説明変数の指定に誤りがあります")
    # return False
if input_list_resVarNames == [] or (set(input_rawDF.columns.tolist()) == False):
    warnings.warn(f"問題変数の指定に誤りがあります")
    # return False

function_names :list[str] = list(set(input_rawDF["functionName"].tolist()))
if len(function_names) == 0:
    warnings.warn("与えられたDFに関数名がありません")
    # return False

result_series_list :list[pd.Series] = []

for function_name in function_names:
    # 関数ごとの生データ
    input_rawDF_per_function = input_rawDF[input_rawDF["functionName"] == function_name]
    # モデルの構築
    models = Models(
        inputDF = input_rawDF_per_function,
        expVarColNames=input_list_expVarNames,
        resVarColNames=input_list_resVarNames,
        targetDF=None,
        modelNames=model_names,
    )

    models.setUpDataBeforeCalcLr()
    models.calcLr()
    models.calcWeightedMAPE()

    dictCalcedWeightedMAPE = models.returnCalculatedWeightedMAPE()

    # 算出されたMAPEの数値をfloatにする
    for key in dictCalcedMAPE.keys():
        dictCalcedWeightedMAPE[key] = float(dictCalcedWeightedMAPE[key])

    # 関数ごとの結果に格納
    dict_for_series :dict = copy.deepcopy(dictCalcedWeightedMAPE)
    dict_for_series["functionName"] = function_name

    series = pd.Series(dict_for_series)
    result_series_list.append(series)

resultDF :pd.DataFrame = pd.DataFrame(result_series_list)
resultDF = addLowestMAPEColumn(inputDF=resultDF, model_name_list=model_names, version=2)


/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estima

In [8]:

resultDF

,modelLin,modelIp,modelLog,modelProcessDividedByProblemSize,modelProblemSizeDividedByProcess,modelLinearSumOf2elementCombination,modelLinearSumOfElementCombinations,modelLinAndIp,modelLinAndLog,modelIpAndLin,modelIpAndLog,modelLogAndLin,modelLogAndIp,functionName,最低値
0,4.972507e-16,6.351914e-01,0.384580,7.691018e-01,7.208057e-01,0.401861,6.762962e-15,0.635191,0.384580,2.123102e-17,3.845805e-01,1.107706e-16,0.635191,void_CommSyncPosVel(Domain,2.123102e-17
1,9.453149e-01,3.144336e-07,0.615327,9.827857e-01,5.219435e-01,0.966441,3.395252e-07,0.945315,0.945315,3.144336e-07,3.144336e-07,6.153269e-01,0.615327,void_VerifyAndWriteFinalOutput(Real_t_Domain,3.144336e-07
2,0.000000e+00,0.000000e+00,0.000000,2.308030e-10,1.862710e-10,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,void_Domain::SetupBoundaryConditions(Int_t),0.000000e+00
3,0.000000e+00,0.000000e+00,0.000000,2.308030e-10,1.862710e-10,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,void_Domain::SetupSymmetryPlanes(Int_t),0.000000e+00
4,0.000000e+00,0.000000e+00,0.000000,2.308030e-10,1.862710e-10,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,MPI_Finalize(),0.000000e+00
5,4.972507e-16,6.351914e-01,0.384580,7.691018e-01,7.208057e-01,0.401861,6.762962e-15,0.635191,0.384580,2.123102e-17,3.845805e-01,1.107706e-16,0.635191,void_CommMonoQ(Domain,2.123102e-17
6,0.000000e+00,0.000000e+00,0.000000,2.308030e-10,1.862710e-10,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,.TAU_application,0.000000e+00
7,0.000000e+00,0.000000e+00,0.000000,2.308030e-10,1.862710e-10,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,MPI_Reduce(),0.000000e+00
8,0.000000e+00,0.000000e+00,0.000000,2.308030e-10,1.862710e-10,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,void_Domain::~Domain(),0.000000e+00
9,1.254224e+00,1.415485e+00,1.362183,1.406233e+00,1.341341e+00,0.776293,4.962105e-01,1.415485,1.362183,1.254224e+00,1.362183e+00,1.254224e+00,1.415485,Real_t_CalcElemVolume(const,4.962105e-01


In [9]:
resultDF.mean()

/tmp/ipykernel_11263/2794288614.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  resultDF.mean()


modelLin                               0.087942
modelIp                                0.284129
modelLog                               0.209925
modelProcessDividedByProblemSize       0.370852
modelProblemSizeDividedByProcess       0.337622
modelLinearSumOf2elementCombination    0.199110
modelLinearSumOfElementCombinations    0.018873
modelLinAndIp                          0.313909
modelLinAndLog                         0.220870
modelIpAndLin                          0.057056
modelIpAndLog                          0.189664
modelLogAndLin                         0.075889
modelLogAndIp                          0.302322
最低値                                    0.018873
dtype: float64

In [10]:
DF_funcName_lowestMAPE :pd.DataFrame = resultDF.filter(items=["functionName", "最低値"])
DF_funcName_lowestMAPE = DF_funcName_lowestMAPE.rename(columns={"functionName":"関数名", "最低値":"最低重み付きMAPE"})

print(DF_funcName_lowestMAPE.style.format(precision=2).to_latex(
    caption=f"ベンチマークプログラムLULESHの関数における最低MAPE",
    label=f"{date}lowestMAPEinLULESH"
))

\begin{table}
\caption{ベンチマークプログラムLULESHの関数における最低MAPE}
\label{2022年6月1日lowestMAPEinLULESH}
\begin{tabular}{llr}
 & 関数名 & 最低重み付きMAPE \\
0 & void_CommSyncPosVel(Domain & 0.00 \\
1 & void_VerifyAndWriteFinalOutput(Real_t_Domain & 0.00 \\
2 & void_Domain::SetupBoundaryConditions(Int_t) & 0.00 \\
3 & void_Domain::SetupSymmetryPlanes(Int_t) & 0.00 \\
4 & MPI_Finalize() & 0.00 \\
5 & void_CommMonoQ(Domain & 0.00 \\
6 & .TAU_application & 0.00 \\
7 & MPI_Reduce() & 0.00 \\
8 & void_Domain::~Domain() & 0.00 \\
9 & Real_t_CalcElemVolume(const & 0.50 \\
10 & MPI_Isend() & 0.04 \\
11 & MPI_Comm_size() & 0.00 \\
12 & MPI_Allreduce() & 0.00 \\
13 & void_CommSend(Domain & 0.00 \\
14 & void_Domain::CreateRegionIndexSets(Int_t_Int_t) & 0.00 \\
15 & void_CalcKinematicsForElems(Domain & 0.00 \\
16 & MPI_Barrier() & 0.00 \\
17 & MPI_Waitall() & 0.00 \\
18 & MPI_Init() & 0.00 \\
19 & void_Domain::SetupCommBuffers(Int_t) & 0.00 \\
20 & void_InitMeshDecomp(Int_t_Int_t_Int_t & 0.00 \\
21 & MPI_Wait() & 0.04 \

# 2022年5月31日～

NPBで重み付きMAPEを出す



In [11]:
def returnAverageWeightedMAPEfromConditions(
    list_benchmarkNames: list[str],
    list_problemSizes: list[str],
    list_processes: list[int],
    list_modelNames: list[str],
) -> pd.DataFrame:
    """returnAverageWeightedMAPEfromConditions
    指定した条件から下記のような表を作成する関数
    | <ベンチマーク名> | <平均重み付きMAPE> |
    Args:
        list_benchmarkNames (list[str])
        list_problemSizes (list[str])
        list_processes (list[int])
        list_modelNames (list[str])
    Returns:
        pd.DataFrame : 上記の説明通りのデータフレーム
        int : なんらかの障害により実行できなくなったとき
    """

    if len(list_benchmarkNames) == 0:
        warnings.warn("ベンチマークプログラム名が指定されていません")
        return -1
    if len(list_problemSizes) == 0:
        warnings.warn("問題サイズが指定されていません")
        return -1
    if len(list_processes) == 0:
        warnings.warn("プロセス数が指定されていません")
        return -1
    if len(list_modelNames) == 0:
        warnings.warn("モデル名が指定されていません")
        return -1

    list_result_col_benchmarkNames: list[str] = []
    list_result_col_averageMAPE: list[float] = []
    dict_resultForDF: dict[list] = {}

    # ベンチマークプログラム名でループ
    for benchmarkName in list_benchmarkNames:
        # 生データを取得
        rawDF :pd.DataFrame = return_rawDF_with_init_param(
            benchmark_name=benchmarkName,
            classes = list_problemSizes,
            processes = list_processes,
            csv_dir_path="./csv_files/",
        )

        _MAPEtable = returnMAPEDFFromRawDF(
            input_rawDF=rawDF, model_names_list=list_modelNames
        )

        _averageMAPEfrom_MAPEtable = returnAverageMAPEfromDF(input_DF = _MAPEtable)
        list_result_col_benchmarkNames.append(benchmarkName)
        list_result_col_averageMAPE.append(_averageMAPEfrom_MAPEtable)

    dict_resultForDF["benchmarkName"] = list_result_col_benchmarkNames
    dict_resultForDF["averageMAPE"] = list_result_col_averageMAPE

    return pd.DataFrame(data=dict_resultForDF)

def test_returnAverageWeightedMAPEfromDF():
    """test_returnAverageWeightedMAPEfromDF()
    
    returnAverageMAPEfromDF()のテスト

    """

    functionName :list[str] = ["func_a", "func_b", "func_c"]
    model_1: list[int] = []


In [12]:
benchmark_names: list[str] = [
    "cg",
    "ep",
    # "ft",
    "is",
    # "lu",
    "mg",
]
problem_sizes: list[str] = ["A", "B", "C", "D", "E", "F"]
processes: list[int] = [2, 4, 8, 16, 32, 64, 128, 256]
model_names: list[str] = [
    "modelLin",
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations"
    # "modelBasicTree",
]

date: str = "2022年5月31日"

warnings.simplefilter("ignore")
latex_table: str = (
    returnAverageWeightedMAPEfromConditions(
        list_benchmarkNames=benchmark_names,
        list_problemSizes=problem_sizes,
        list_processes=processes,
        list_modelNames=model_names,
    )
    .style.set_precision(2)
    .to_latex(
        caption=f"平均MAPE_table1",
        label=f"{date}allModelsMAPE",
    )
)

print(latex_table)

\begin{table}
\caption{平均MAPE_table1}
\label{2022年5月31日allModelsMAPE}
\begin{tabular}{llr}
 & benchmarkName & averageMAPE \\
0 & cg & 6.59 \\
1 & ep & 0.00 \\
2 & is & 2.23 \\
3 & mg & 0.88 \\
\end{tabular}
\end{table}



# 2022年5月31日

関数 `Real t CalcElemVolume(const` のMAPEが大きい、つまり精度が低いのでプロットなどをして確認する


In [13]:
date :str = "2022年5月31日"

input_rawDF :pd.DataFrame = return_rawDFinLULESH(
    processes=lulesh_processes,
    iterations=lulesh_iterations,
    sizes=lulesh_sizes,
    csvDirPath=csvDirPath,
)

input_rawDF = input_rawDF.rename(columns={"Name":"functionName"})

input_list_expVarNames :list[str] = ["process", "iteration", "size"]
input_list_resVarNames :list[str] = ["#Call"]

model_names: list[str] = [
    "modelLin",
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations"
    # "modelBasicTree",
]

targetDF = input_rawDF[input_rawDF["functionName"] == "Real_t_CalcElemVolume(const"]
targetDF = targetDF.sort_values("#Call")


In [14]:
print(targetDF.to_csv())

,functionName,#Call,process,iteration,size
5,Real_t_CalcElemVolume(const,36864.0,8,8,16
8,Real_t_CalcElemVolume(const,36864.0,125,8,16
10,Real_t_CalcElemVolume(const,36864.0,27,8,16
8,Real_t_CalcElemVolume(const,36864.0,216,8,16
8,Real_t_CalcElemVolume(const,36864.0,343,8,16
8,Real_t_CalcElemVolume(const,36864.0,512,8,16
8,Real_t_CalcElemVolume(const,36864.0,64,8,16
8,Real_t_CalcElemVolume(const,69632.0,343,16,16
8,Real_t_CalcElemVolume(const,69632.0,512,16,16
6,Real_t_CalcElemVolume(const,69632.0,8,16,16
8,Real_t_CalcElemVolume(const,69632.0,27,16,16
13,Real_t_CalcElemVolume(const,69632.0,125,16,16
8,Real_t_CalcElemVolume(const,69632.0,216,16,16
9,Real_t_CalcElemVolume(const,69632.0,64,16,16
8,Real_t_CalcElemVolume(const,124416.0,216,8,24
6,Real_t_CalcElemVolume(const,124416.0,8,8,24
8,Real_t_CalcElemVolume(const,124416.0,512,8,24
8,Real_t_CalcElemVolume(const,124416.0,125,8,24
8,Real_t_CalcElemVolume(const,124416.0,64,8,24
8,Real_t_CalcElemVolume(const,124416.0,27,8,24
8,Real_t_CalcE

# 2022年4月17日～

次のような表を作成する

採用される割合 (MAPE の最大値 [%] ，MAPE の最小値 [%]) [%]

| ベンチマークプログラム名 | 線形モデル               | 対数モデル               | 反比例モデル              |
|--------------|---------------------|---------------------|---------------------|
| str          | float(float, float) | float(float, float) | float(float, float) |


目標となるのは一気にこのベンチマークプログラムを作成することだが、既存のライブラリ関数などを利用し、まずはベンチマークごとに作成可能にする。

メモ

## 実装予定

1. 行方向に最小値を検出
2. 最小値以外をNaNに変更
3. 列方向に最小値と最大値を検出

## 
